### Качаем данные [(ссылка)](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words)

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/docword.nips.txt.gz

--2015-08-04 18:16:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/docword.nips.txt.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.1.95
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.1.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2288169 (2,2M) [application/x-gzip]
Saving to: `docword.nips.txt.gz'

100%[======================================>] 2 288 169    924K/s   in 2,4s    

2015-08-04 18:16:56 (924 KB/s) - `docword.nips.txt.gz' saved [2288169/2288169]



In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/vocab.nips.txt

--2015-08-04 18:16:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/vocab.nips.txt
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.1.95
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.1.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100807 (98K) [text/plain]
Saving to: `vocab.nips.txt'

100%[======================================>] 100 807      162K/s   in 0,6s    

2015-08-04 18:16:59 (162 KB/s) - `vocab.nips.txt' saved [100807/100807]



разархивируем 

In [3]:
!gzip -d docword.nips.txt.gz

смотрим 

In [4]:
!head docword.nips.txt

1500
12419
746316
1 2 1
1 39 1
1 42 3
1 77 1
1 95 1
1 96 1
1 105 1


In [5]:
!head vocab.nips.txt

a2i
aaa
aaai
aapo
aat
aazhang
abandonment
abbott
abbreviated
abcde


### Делаем батчи

Для этого подготовим отдельную папку 'uci_data' и положим в неё файлы 'docword.nips.txt', 'vocab.nips.txt'. Названия файлов должны быть в формате 'docword.collection_name.txt', 'vocab.collection_name.txt'

In [6]:
import artm
from artm.artm_model import parse

In [7]:
!mkdir uci_data

In [8]:
!ls

docword.nips.txt  Tutorial UCI bag of words.ipynb  uci_data  vocab.nips.txt


In [9]:
!mv docword.nips.txt uci_data

In [10]:
!mv vocab.nips.txt uci_data

In [11]:
!ls

Tutorial UCI bag of words.ipynb  uci_data


В команду parse передаём название коллекции, путь к папке где лежат данные и формат данных. 

In [12]:
parse(collection_name='nips', data_path='uci_data', data_format = 'bow_uci')

В результате вызова создастся папка collection_name, где будут находится батчи и словарь. 

In [13]:
!cd nips && ls

16c498f3-aaf9-4950-b15b-044b2a6338fc.batch  dictionary
469a3037-ec94-44dd-a9d9-0e9fb5d22508.batch


### Строим модель 

In [14]:
model = artm.artm_model.ArtmModel(num_topics=20)

На этапе инициализации передаём путь к словарю 

In [15]:
model.load_dictionary(dictionary_name='dictionary', dictionary_path='nips/dictionary')
model.initialize(dictionary_name='dictionary')

In [16]:
model.fit_offline(data_path='nips', num_collection_passes=2)

In [17]:
def show_topics(model, top):
    phi = model.get_phi()
    for topic in model.topic_names:
        phi = phi.sort(topic, ascending=False)
        top_words = ''
        for word in phi[topic][:top].index:
            top_words += word + ', '
        print topic
        print top_words + '\n'

def show_topics_with_highlihts(model, top, words):
    phi = model.get_phi()
    for topic in model.topic_names:
        phi = phi.sort(topic, ascending=False)
        top_words = ''
        for word in phi[topic][:top].index:
            if word in words:
                word = '\x1b[93;46m' + word + '\x1b[0m'
            top_words += word + ', '
        print topic
        print top_words + '\n'

Смотрим на получившиеся темы 

In [18]:
show_topics(model, 20)

@topic_0
network, learning, set, neural, model, algorithm, input, weight, data, function, space, result, neuron, number, parameter, vector, method, performance, values, point, 

@topic_1
network, weight, system, model, neuron, output, neural, algorithm, set, learning, parameter, layer, result, current, input, error, problem, step, chip, dynamic, 

@topic_2
network, output, unit, function, problem, data, learning, layer, error, model, training, point, vector, result, net, pattern, mean, algorithm, field, system, 

@topic_3
model, network, neuron, system, data, input, unit, neural, result, equation, control, parameter, number, order, weight, function, synaptic, cell, training, pattern, 

@topic_4
network, system, data, algorithm, model, parameter, training, neural, term, layer, weight, linear, word, cell, set, unit, probability, single, point, space, 

@topic_5
data, neural, model, result, number, weight, set, system, neuron, training, space, rate, pattern, error, level, algorithm, cell,

In [19]:
interesting_words = ['network', 'neural', 'recognition', 'image', 'pattern']
show_topics_with_highlihts(model, 20, interesting_words)

@topic_0
network, learning, set, neural, model, algorithm, input, weight, data, function, space, result, neuron, number, parameter, vector, method, performance, values, point, 

@topic_1
network, weight, system, model, neuron, output, neural, algorithm, set, learning, parameter, layer, result, current, input, error, problem, step, chip, dynamic, 

@topic_2
network, output, unit, function, problem, data, learning, layer, error, model, training, point, vector, result, net, pattern, mean, algorithm, field, system, 

@topic_3
model, network, neuron, system, data, input, unit, neural, result, equation, control, parameter, number, order, weight, function, synaptic, cell, training, pattern, 

@topic_4
network, system, data, algorithm, model, parameter, training, neural, term, layer, weight, linear, word, cell, set, unit, probability, single, point, space, 

@topic_5
data, neural, model, result, number, weight, set, system, neuron, training, space, rate, pattern, error, level, algorithm, cell,